In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
%matplotlib inline

C:\Users\kfarkas\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
data = '../practice_data/default_recovered.csv'

df = pd.read_csv(data)

In [7]:
df.head()

,ID,Outstanding _principal,Prime,Joint,PV,ABP,DP,12_mo_inquires,Rev_over_90,Auto_App,BA_V,Grade,IDA,BK,Deceased,BA_change_pre _issue,BA_change_post_issue,Days_between_issue_default,Recovered
0,1,8476.17,1,0,2,0,0,0,0,1,0,C,314,0,0,0,0,112,1
1,2,31370.35,1,0,2,0,0,0,0,1,0,C,422,0,0,0,0,146,0
2,3,11727.27,1,0,2,0,0,0,0,0,0,C,364,0,0,0,0,124,0
3,4,3764.33,0,0,2,0,0,0,0,0,0,G,353,0,0,0,1,117,0
4,5,10000.00,1,0,3,0,0,0,0,0,0,B,412,0,0,0,1,48,0


In [8]:
df.corr()

,ID,Outstanding _principal,Prime,Joint,PV,ABP,DP,12_mo_inquires,Rev_over_90,Auto_App,BA_V,IDA,BK,Deceased,BA_change_pre _issue,BA_change_post_issue,Days_between_issue_default,Recovered
ID,1.000000,0.013632,0.042180,0.025128,-0.013614,0.015559,0.088895,-0.040398,-0.017824,0.018205,0.010500,0.024575,-0.050274,0.007856,0.029480,0.009156,-0.502294,0.034817
Outstanding _principal,0.013632,1.000000,0.369037,0.043790,-0.031327,0.013453,-0.019797,0.019884,-0.052529,0.044186,-0.008415,-0.044825,0.046258,-0.010412,0.015046,-0.013809,-0.014989,-0.058900
Prime,0.042180,0.369037,1.000000,-0.006254,-0.000160,0.011036,-0.055544,0.023334,-0.170617,0.095166,0.008013,-0.013674,0.025100,0.035899,0.000106,0.035778,-0.013284,0.018751
Joint,0.025128,0.043790,-0.006254,1.000000,0.003328,-0.007274,-0.010046,-0.027859,0.027572,-0.063241,0.022691,0.044325,-0.000920,-0.010591,0.026268,0.018532,-0.013822,0.006337
PV,-0.013614,-0.031327,-0.000160,0.003328,1.000000,-0.078097,-0.026727,-0.004065,-0.027152,0.053656,0.127384,0.096082,-0.013249,-0.000808,0.014501,0.011125,0.015129,-0.005091
ABP,0.015559,0.013453,0.011036,-0.007274,-0.078097,1.000000,0.000663,-0.021735,0.020552,-0.043199,-0.032003,0.082687,-0.008513,0.006836,-0.009885,-0.085516,-0.120678,0.085334
DP,0.088895,-0.019797,-0.055544,-0.010046,-0.026727,0.000663,1.000000,0.012840,0.011746,-0.026176,-0.013088,0.001862,0.009588,-0.004384,-0.008486,0.013371,-0.061811,-0.014191
12_mo_inquires,-0.040398,0.019884,0.023334,-0.027859,-0.004065,-0.021735,0.012840,1.000000,-0.075712,0.014629,0.000963,-0.025242,-0.017899,-0.017599,-0.004745,0.025386,0.018465,-0.030067
Rev_over_90,-0.017824,-0.052529,-0.170617,0.027572,-0.027152,0.020552,0.011746,-0.075712,1.000000,-0.009471,-0.014625,0.010906,-0.031749,-0.002117,0.039145,-0.024628,-0.000065,0.002085
Auto_App,0.018205,0.044186,0.095166,-0.063241,0.053656,-0.043199,-0.026176,0.014629,-0.009471,1.000000,-0.082395,-0.158430,0.005662,-0.008091,-0.027908,0.001239,0.048014,0.013285
